In [1]:
import pandas as pd
import os
import numpy as np
# ^^^ pyforest auto-imports - don't write above this line
import math
import os
import pandas as pd
import numpy as np
import datetime
import sklearn.tree

from IPython.display import SVG
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import RFE, SelectKBest, f_classif, mutual_info_classif, chi2,SelectPercentile,SelectFdr,SelectFpr,SelectFromModel
import os

from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.dummy import DummyClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats as stats

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

# Preliminary Data Import
- I am going to import 2 dfs, one that had the data (with no column names) and another that contains the correct column names (necessary for analysis ;)) 
- I am going to read the larger df in by chunks, since this df is over 6,000,000 rows and won't import in one fell swoop onto my machine
- Each chunk will replace its column names with the proper names
- Each chunk will be truncated into only information pertinent to the AB analysis, primarily:
    - XpId
    - XpGroupType
    - XpUrl
    - Id
    - SureyRating
    - ProcessSessionId
    - Platform
    - Product
    - SurveyRatingQuestion
- After this data import, I will be translating the questions into english to use in tagging NPS questions as suite or app

In [4]:
#set proper working directors
os.chdir('C:\\Users\\paperspace\\Desktop\\Microsoft\\AB Testing')

#import data (only first n00k rows as a sample)
chunksize = 100000
df = pd.read_csv('end_user_nps_cy2020_only_e-exp_partial.tsv', delimiter = '\t', chunksize = chunksize)

#read in dataset that contains column names
colnames = pd.read_excel('sample_AB.xlsx', nrows = 1)

df.columns = colnames.columns

#read in df by chunks
chunks = []
for idx, chunk in enumerate(df):
    chunk.columns = colnames.columns
    chunk = chunk[['XpId', 'XpGroupType', 'XpUrl', 'Id','SurveyId', 'SurveyRating', 'ProcessSessionId', 'Platform', 'Product']]
    chunks.append(chunk)
    print(f'finished importing chunk {idx + 1}')

#concatenate chunks in list, have to pass sort=False to ignore the FutureWarning produced by this
df = pd.concat(chunks, sort = False)

finished importing chunk 1
finished importing chunk 2
finished importing chunk 3


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (30,170,181) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 4
finished importing chunk 5


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (170,181) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 6


...

finished importing chunk 58


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,170,181,205,254) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 59
finished importing chunk 60
finished importing chunk 61
finished importing chunk 62
finished importing chunk 63


C:\Users\paperspace\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (8,205) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


finished importing chunk 64


In [ ]:
#send df to outfile
df.to_csv('Stage1ABdata.csv', index = False)

In [1]:
df = pd.read_csv('Stage1ABdata.csv')

In [834]:
df.head()

,XpId,XpGroupType,XpUrl,Id,SurveyId,SurveyRating,ProcessSessionId,Platform,Product
0,38234,T,https://ecs.skype.com/?page=ExperimentPage&id=38234,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel
1,42370,T,https://ecs.skype.com/?page=ExperimentPage&id=42370,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel
2,42371,T,https://ecs.skype.com/?page=ExperimentPage&id=42371,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel
3,44777,T,https://ecs.skype.com/?page=ExperimentPage&id=44777,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel
4,38235,T,https://ecs.skype.com/?page=ExperimentPage&id=38235,flnps_v2_7c12a533885454a19ad54a76b7f80646,092b8d95-cf3f-4172-a11b-f5bbc03077ac,5.0,ea85437b-3cea-4a30-b590-f41a3258cf1f,Mac,Mac Powerpoint


# Tagging NPS Question Types
- I am going to be finding out whether a user was asked about the full suite of Office 365 Products, or whether they were asked about a specific app, I will then use that information to tag each survey answer, which I can then use to split the data into "suite" and "app" subsets. Once the data is filtered, I will then feed each of these chunks into the function below to generate a report for A/B testing. I am also going to be feeding in the full dataframe, not separated by O365 and application questions, to show what the impact of the experiments were on the overall data. 
- Tested this with Sensei/DM as well

In [ ]:
#using Google Translate to translate foreign language questions into english (this may not work on some VPNs, so if you are having trouble with this
# then I recommend you change VPNs)
def translate_text(text, dest_language="en"):
        # Used to translate using the googletrans library
    import json
    translator = Translator()
    try:
        translation = translator.translate(text=text, dest=dest_language)
    except json.decoder.JSONDecodeError:
        # api call restriction
        process = subprocess.Popen(["nordvpn", "d"], stdout=subprocess.PIPE)
        process.wait()
        process = subprocess.Popen(["nordvpn", "c", "canada"], stdout=subprocess.PIPE)
        process.wait()
        return Process_Data.translate_text(text=text, dest_language=dest_language)
    return translation

#translate all questions
question_list = [x for x in questions.Question]

translated_questions = [translate_text(x).text for x in question_list]

questions.insert(loc = 1, column = 'TranslatedQuestions', value = [question for question in translated_questions])

questions.to_csv('translated_questions_from_NPS_survey.csv', index = False)

In [ ]:
translated_questions = pd.read_csv('translated_questions._from_NPS_survey.csv')
#merging translated questions into original df, dropping redundant question column 
df = translated_questions.join(df.set_index(df.SurveyRatingQuestion), on = 'Question').drop(['Question'], axis = 1)

In [ ]:
#searching for keywords in the df and separating products out by keyword -- this generates some duplicates becuase some questions include BOTH the
# name of an app (i.e. Word) AND the word "app" as well, so we need to filter out duplicates with boolean logic after concatenating all of them, then filter 
# -- Therea are about 20 surveys that we will not capture filtering out in this manner that refer to "games" or other things that were simply lost in 
#translation and unable to be tied back to a specific product
products = ['365|Office|office', 
            "application|apli|App|app", 
            'Word|word', 'Excel|excel', 
            'Powerpoint|powerpoint|PowerPoint', 'Outlook|outlook', 'Access|access', 'Visi|visi', 
            'OneNote', 'program', 'Proj|proj']

d = {}
for name in products:
    d[name] = pd.DataFrame(df[(df['TranslatedQuestion'].astype(str).str.contains(f'{name}')) | (df['SurveyRatingQuestion'].astype(str).str.contains(f'{name}'))])

#concatenating dfs together to see exactly how many survey responses we didn't capture using the above filtering
dfs = []
for x in d.keys():
    prod = pd.DataFrame(d[x])
    dfs.append(prod)

    
nps_df = pd.concat(dfs, axis = 0).drop_duplicates()

In [1537]:
#filter out cases that have NPS questions pertaining to the suite and speicific apps
suite = nps_df[nps_df['TranslatedQuestion'].str.contains('365|Office|office|offi|ofi')]

app = nps_df[~nps_df.index.isin(suite.index)]

#add in a tag 
suite.insert(loc = 0, column = 'NPS Tag', value = 'O365 Suite')
app.insert(loc = 0, column = 'NPS Tag', value = 'App')

nps_df.to_csv('NPS_data_separated_by_suite_and_app.csv', index = False)

#set proper working directors
os.chdir('C:\\Users\\fulto\\Desktop\\AB Testing\\AB Test Scripts')
suite_and_app_df = pd.read_csv('NPS_data_separated_by_suite_and_app.csv')

In [843]:
#look at the head of the data, this is what it should look like to be fed properly into the scoring function
suite_and_app_df.head()

,XpId,XpGroupType,XpUrl,Id,SurveyId,SurveyRating,SurveyRatingQuestion,ProcessSessionId,Platform,Product,NPS Tag
0,38234,T,https://ecs.skype.com/?page=ExperimentPage&id=38234,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,How likely are you to recommend Office 365 to a friend or colleague?,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel,O365 Suite
1,42370,T,https://ecs.skype.com/?page=ExperimentPage&id=42370,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,How likely are you to recommend Office 365 to a friend or colleague?,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel,O365 Suite
2,42371,T,https://ecs.skype.com/?page=ExperimentPage&id=42371,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,How likely are you to recommend Office 365 to a friend or colleague?,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel,O365 Suite
3,44777,T,https://ecs.skype.com/?page=ExperimentPage&id=44777,flnps_v2_b7963972fdae58f698feb84fe9deb246,092b8d95-cf3f-4172-a11b-f5bbc03077ac,1.0,How likely are you to recommend Office 365 to a friend or colleague?,1f923701-08bc-4537-9a2e-db4a17624445,Android,Android Excel,O365 Suite
4,38235,T,https://ecs.skype.com/?page=ExperimentPage&id=38235,flnps_v2_7c12a533885454a19ad54a76b7f80646,092b8d95-cf3f-4172-a11b-f5bbc03077ac,5.0,How likely are you to recommend Office 365 to a friend or colleague?,ea85437b-3cea-4a30-b590-f41a3258cf1f,Mac,Mac Powerpoint,O365 Suite


In [847]:
#check on questions again to see if it looks right -- it does
#suite_and_app_df.groupby(['NPS Tag']).SurveyRatingQuestion.value_counts()

In [1538]:
#separate out suite and app dfs
suite_df = suite_and_app_df[suite_and_app_df['NPS Tag'] == 'O365 Suite'].drop('NPS Tag', axis = 1)
app_df = suite_and_app_df[suite_and_app_df['NPS Tag'] == 'App'].drop('NPS Tag', axis = 1)

## Creating Function To Run A/B Test
- This function will run the entire analysis in one go
- details are commented out for eachn line of code
- esentially this takes the df we pulled above, calculates the Standard Error of the NPS for the treatment and control groups, calculates the Margin of Error at 95% CI, then creates upper and lower bounds (CIs) fot the treatment and control groups, then finally determines if a flight was statistically significant. 
- All of this information is fed into two final output dataframes, one for platform aggregated info and the other for product specific tests
- Beware that there are some flights that have no treatment group and visa versa, sso I filtered the data further to ONLY contain treatment-control pairs, (experiments that contained both control AND treatment groups). From here, I also filtered the data to only contain treatment-control pairs where there were greater than 30 individuals (60 total) in each group. The resulting dataframe is a fraction of the total number of experiments, but contains no solitary groups and no underpowered samples (some of the samples only had one person in them, which also generates "infinite" answers, so be on the lookout for that as well. 

In [1544]:
%%time

def run_ab_test(df):

    '''
    
    This function takes an input df, using these columns from NPS data ('XpId', 'XpGroupType', 'XpUrl', 'Id', 'SurveyRating', 'ProcessSessionId', 'Platform', 'Product'), 
    It then takes this df and scores it for MOE and statsig, outputs two final dfs, one of them that is aggregated at the experiment level by product, and another that 
    has this information broken down for every product included in every experiment
    
    Arguments: one df that has the columns from NPS data specified in the sentence above this
    
    Calling the Function: calling this function requires tuple unpacking (i.e. df1, df2 = fun_ab_test(df))
    
    '''
    print('---->---->Creating and Scoring Data Frame for Experiments Aggregated By Product<----<-----')
    print(' ')
    print('----->----->Scoring Control Group----->----->')

    #rename columns to more intuitive names
    df = df.rename(columns = {'XpId':'ExperimentId', 'XpUrl':'ExperimentUrl', 'XpGroupType':'ExperimentGroup', 'Id':'SurveyId', 'XpUrl':'ExperimentUrl', 'SurveyRating':'NPS'})

    #map new scores to NPS --> 1-3 = -100, 4 = 0, 5 = 100
    df['NPS'] = df['NPS'].map({1:-100, 2:-100, 3:-100, 4:0, 5:100})
    
    print(' ')

    #creating groupby data frame and then unstacking it -- this is the primary meat of this analysis
    score_df = pd.DataFrame(df.groupby(['ExperimentId','ExperimentGroup']).agg({'NPS':np.mean, 'ExperimentGroup':'size'}).unstack())

    #rename column from multi index to single index
    score_df.columns = ['NPS Control', 'NPS Treatment', 'Control Count', 'Treatment Count']

    #fill na values with 0 to indicate that there was nobody in a specific group
    score_df = score_df.fillna(0).reset_index()

    #create column for difference in scores
    score_df['Difference (T - C)'] = score_df['NPS Treatment'] - score_df['NPS Control']
    #score_df.head(15)

    #get # of passive, promoters, and detractors for each control group
    control = df[df['ExperimentGroup'] == 'C'].groupby(['ExperimentId', 'NPS']).agg({'NPS':'size'}).unstack()

    #rename columns
    control.columns = ['Detractors', 'Passives', 'Promoters']

    #create total N size for control groups
    control['Total N'] = control.sum(1)

    #fill na values with 0 since this indicates that there was no score present, also reset the index to make sure experiment ID 
    #is a column
    control = control.fillna(0).reset_index()

    #find out which experiments did not have control groups
    no_control_groups = []
    for x in list(score_df.ExperimentId):
        if x not in list(control.ExperimentId):
            no_control_groups.append(x)

    #add those experiments to the control group df, we will then sort the df in order of experiment number 
    new_rows = []
    for x in no_control_groups:
        row = pd.Series([x,0,0,0,0], index=['ExperimentId', 'Detractors', 'Passives', 'Promoters', 'Total N'])
        new_rows.append(row)
    
    #create new df from no control rows
    new_rows_df = pd.DataFrame(new_rows)

    #concatenate the new rows (no control group) with the control group df
    control = pd.concat([control, new_rows_df], axis = 0).sort_values(by = 'ExperimentId').reset_index(drop = True)

    #sort our dfs by experiment so they are in the same order now
    control = control.sort_values('ExperimentId')
    score_df = score_df.sort_values('ExperimentId')

    #now we can add in the control group NPS to the correct experiment (there will be no score for exps that didn't have a control group)
    control['NPS'] = score_df['NPS Control']

    #creating function for calculating MOE, taken from formula provided in NPS wiki
    def calculate_MOE(nps, promoters, passives, detractors, N):
        '''
        This function calculates Margin of Error for a given treatment or control group
        
        Arguments: (nps = net promoter score for group, promoters|passives|detractors = N size for each subset of a group, N = total N size for the group overall)
        
        '''
        a = (1 - nps)**2
        b = promoters/N
        c = (0 - nps)**2
        d = passives/N
        e = (-1 - nps)**2
        f = detractors/N
        g = N - 1
        moe = 1.96 * np.sqrt((a * b + c * d + e * f) / (g))
        return moe

    #calculating MOE for control group
    control['MOE Control'] = calculate_MOE(control['NPS'], control['Promoters'], control['Passives'], control['Detractors'], control['Total N'])

    #filling in NA values (where there was no control group, so not MOE)
    control.fillna(0, inplace = True)

    #creating column in score_df for the Control MOE to be used in lower and upper confidence interval calculations
    score_df['MOE Control'] = control['MOE Control']

    #creating upper and lower bound calculations for the Control group and adding them as columns to the score df
    score_df['Control Lower Bound'] = score_df['NPS Control'] - score_df['MOE Control']
    score_df['Control Upper Bound'] = score_df['NPS Control'] + score_df['MOE Control']

    print('----->----->Scoring Treatment Group----->----->')
    # #creating treatment df now
    # #get # of passive, promoters, and detractors for each control group
    treatment = df[df['ExperimentGroup'] == 'T'].groupby(['ExperimentId', 'NPS']).agg({'NPS':'size'}).unstack()


    ##rename columns
    treatment.columns = ['Detractors', 'Passives', 'Promoters']


    #create total N size for control groups
    treatment['Total N'] = treatment.sum(1)


    #fill na values with 0 since this indicates that there was no score present, also reset the index to make sure experiment ID 
    #is a column
    treatment = treatment.fillna(0).reset_index()

    #add in NPS score for control group as a column
    #treatment['NPS Treatment'] = score_df['NPS Treatment']


    #find out which experiments did not have treatment groups
    no_treatment_groups = []
    for x in list(score_df.ExperimentId):
        if x not in list(treatment.ExperimentId):
            no_treatment_groups.append(x)

    #add those experiments to the control group df, we will then sort the df in order of experiment number 
    new_treatment_rows = []
    for x in no_treatment_groups:
        row = pd.Series([x,0,0,0,0], index=['ExperimentId', 'Detractors', 'Passives', 'Promoters', 'Total N'])
        new_treatment_rows.append(row)
    
    #create new df from no control rows
    new_treatment_rows_df = pd.DataFrame(new_treatment_rows)

    #concatenate the new rows (no control group) with the control group df
    treatment = pd.concat([treatment, new_treatment_rows_df], axis = 0, sort = False).sort_values(by = 'ExperimentId').reset_index(drop = True)

    #sort our dfs by experiment so they are in the same order now
    treatment = treatment.sort_values('ExperimentId')


    #now we can add in the treatment group NPS to the correct experiment (there will be no score for exps that didn't have a treatment group)
    treatment['NPS Treatment'] = score_df['NPS Treatment']


    #calculating MOE for treatment group using function created eariler
    treatment['MOE Treatment'] = calculate_MOE(treatment['NPS Treatment'], treatment['Promoters'], treatment['Passives'], treatment['Detractors'], 
                                         treatment['Total N'])

    #filling NA values (where there was no treatment group) -- only 5 or 6 that had low N and only control
    treatment.fillna(0, inplace = True)

    #creating column in score_df for the Control MOE to be used in lower and upper confidence interval calculations
    score_df['MOE Treatment'] = treatment['MOE Treatment']

    #creating upper and lower bound calculations for the Control group and adding them as columns to the score df
    score_df['Treatment Lower Bound'] = score_df['NPS Treatment'] - score_df['MOE Treatment']
    score_df['Treatment Upper Bound'] = score_df['NPS Treatment'] + score_df['MOE Treatment']
    
    #create boolean lists for if the lower and upper bounds of the control were greater/less than those of the treatment group
    a = (score_df['Control Upper Bound'] > score_df['Treatment Lower Bound']) & (score_df['Control Lower Bound'] > score_df['Treatment Lower Bound'])
    b = (score_df['Control Upper Bound'] > score_df['Treatment Upper Bound']) & (score_df['Control Lower Bound'] > score_df['Treatment Upper Bound'])
    c = (score_df['Treatment Upper Bound'] > score_df['Control Lower Bound']) & (score_df['Treatment Lower Bound'] > score_df['Control Lower Bound'])
    d = (score_df['Treatment Upper Bound'] > score_df['Control Upper Bound']) & (score_df['Treatment Lower Bound'] > score_df['Control Upper Bound'])
    
    #concatenate control and treatment pairs
    control_statsig = pd.concat([a, b], axis = 1)
    treatment_statsig = pd.concat([c, d], axis = 1)
    
    #must have a pair of True values (either two on left, or two on right) to be statsig
    control_statsig = pd.DataFrame(control_statsig.sum(1) == 2).rename(columns = {0:'StatSig?'})
    control_sig_indices = control_statsig[control_statsig['StatSig?'] == True].index
    
    #must have a pair of True values (either two on left, or two on right) to be statsig
    treatment_statsig = pd.DataFrame(treatment_statsig.sum(1) == 2).rename(columns = {0:'StatSig?'})
    treatment_sig_indices = treatment_statsig[treatment_statsig['StatSig?'] == True].index
    
    #create series so that you can concatenate them next into one column then one list
    treatment_sig_indices = pd.Series([x for x in treatment_sig_indices])
    control_sig_indices = pd.Series([x for x in control_sig_indices])
    
    #get statistically significant index numbers to filter out statsig experiments
    sig_indices = [x for x in pd.concat([treatment_sig_indices, control_sig_indices], axis = 0).reset_index(drop = True)]
    
    #run loop to tag StatSig experiments and create a column for their labels
    statsig = []
    for x in score_df.index:
        if x in sig_indices:
            statsig.append('StatSig')
        else:
            statsig.append('Not')
    score_df['StatSig'] = statsig
    
    #get product and platform for experiments and add them in as columns
    unique_xps = df.drop_duplicates(subset = 'ExperimentId')[['ExperimentId', 'Platform', 'Product']].sort_values('ExperimentId').reset_index(drop = True)
    
    #insert platform and product columns
    score_df.insert(loc=1, column = 'Platform', value = unique_xps['Platform'])
    
    def get_products_in_experiment(df):
        
        '''
        
        This function takes the same df as an input and inserts a column that includes all of the products that an experiment was conducted in
        
        '''
        #group experimnets by thei id and the product
        experiment_products = df.groupby('ExperimentId').Product.value_counts().unstack().reset_index()
        
        #create a dataframe that we can use to tag values that are non-NaN with the name of the product
        a = pd.DataFrame(experiment_products.iloc[:, 1]).fillna(0)
        new = []
        for x in [x for x in a.iloc[:, 0]]:
            if x != 0:
                new.append(", ".join(a.columns.tolist()))
            else:
                new.append(np.nan)
        a[a.columns] = new
        
        #create lest to store names of products in their row
        new = []
        for col in experiment_products.columns:
            a = pd.DataFrame(experiment_products.loc[:, f'{col}']).fillna(0)
            for x in [x for x in a.iloc[:, 0]]:
                if x != 0:
                    new.append(', '.join(a.columns.tolist()))
                else:
                    new.append(np.nan)
                
        # Create a function called "chunks" with two arguments, l and n:
        def chunks(l, n):
            # For item i in a range that is a length of l,
            for i in range(0, len(l), n):
                # Create an index range for l of n items:
                yield l[i:i+n]
        
        #create df that has values replaces with product names, and has 0s otherwise
        new_exp_prod = pd.DataFrame([x for x in chunks(new, experiment_products.shape[0])]).T
        new_exp_prod.columns = experiment_products.columns
        new_exp_prod['ExperimentId'] = experiment_products['ExperimentId']
        new_exp_prod.index = new_exp_prod['ExperimentId']
        
        #create list to store product names
        prod_names = []
        for idx, row in new_exp_prod.fillna(0).iterrows():
            for x in row[1:]:
                if (x != 0) & (x != 'ExperimentId'):
                    prod_names.append(x)
                else:
                    prod_names.append(0)
        #create final list to store product names and throw them into the df
        final_list = []
        for idx, row in new_exp_prod.iterrows():
            final_list.append([x for x in row[1:] if str(x) != 'nan'])
        
        return pd.DataFrame({0:[x for x in final_list]})
    
    #insert these as a column, this is the column that contains all product names for a given experiment ID
    score_df.insert(loc = 1, column = 'Products', value = [x for x in get_products_in_experiment(df).iloc[:, 0]])
    
    #this will get rid of brackets for the column values
    score_df['Products'] = [", ".join(x) for x in score_df['Products']]
    
    #add in the Experiment URL
    score_df.insert(loc = 1, column = 'ExperimentUrl', value = pd.DataFrame(df.sort_values('ExperimentId').ExperimentUrl.unique()))
    
    #filter out experiments that had sample sizes of less than 30 in EITHER the treatment or control groups or if either one of the groups was not present
    score_df = score_df[(score_df['Control Count'] > 30) & (score_df['Treatment Count'] > 30) & ((score_df['Control Count'] != 0) & score_df['Treatment Count'] != 0)]
    
    #reset the index
    score_df = score_df.reset_index(drop = True)
    
    
    
    #Now the function will create a separate scored data frame for each experiment, but broken down by product
    print(' ')
    print('---->---->Creating and Scoring Each Experiment, Broken Down By Product, Starting With Control Group<----<----')
    print(' ')
    print('---->---->Scoring Control Groups---->---->')
    print(' ')
    
    #rename columns to more intuitive names
    df = df.rename(columns = {'XpId':'ExperimentId', 'XpUrl':'ExperimentUrl', 'XpGroupType':'ExperimentGroup', 'Id':'SurveyId', 'XpUrl':'ExperimentUrl', 'SurveyRating':'NPS'})

    #map new scores to NPS --> 1-3 = -100, 4 = 0, 5 = 100
    #df['NPS'] = df['NPS'].map({1:-100, 2:-100, 3:-100, 4:0, 5:100})

    all_prods = pd.DataFrame(df.groupby(['ExperimentId','Platform', 'Product', 'ExperimentGroup']).agg({'NPS':np.mean, 'ExperimentGroup':'size'}).unstack()).reset_index()

    all_prods.fillna(0, inplace = True)

    #rename columns
    all_prods.columns = ['ExperimentId', 'Platform', 'Product', 'NPS Control', 'NPS Treatment', 'ControlSize', 'TreatmentSize']

    all_prods = all_prods[(all_prods['ControlSize'] > 30) & (all_prods['TreatmentSize'] > 30)]

    all_prods['Difference (T - C)'] = all_prods['NPS Treatment'] - all_prods['NPS Control']

    #filter out for control groups
    all_prods_control = df[df['ExperimentGroup'] == 'C'].groupby(['ExperimentId','Product', 'NPS']).agg({'NPS':'size'}).unstack()

    all_prods_control.fillna(0, inplace = True)

    #rename columns
    all_prods_control.columns = ['Detractors', 'Passives', 'Promoters']

    #create total N size for control groups
    all_prods_control['Total N'] = all_prods_control.sum(1)

    all_prods_control = all_prods_control.fillna(0).reset_index()

    all_prods_control = all_prods_control[(all_prods_control['Total N'] > 30)]

    no_control_groups = []
    for x in list(all_prods.ExperimentId):
        if x not in list(all_prods_control.ExperimentId):
            no_control_groups.append(x)

    new_rows = []
    for x in no_control_groups:
        row = pd.Series([x,0,0,0,0,0], index=['ExperimentId', 'Product', 'Detractors', 'Passives', 'Promoters', 'Total N'])
        new_rows.append(row)

    new_rows_df = pd.DataFrame(new_rows)

    #concatenate the new rows (no control group) with the control group df
    all_prods_control = pd.concat([all_prods_control, new_rows_df], axis = 0).sort_values(by = 'ExperimentId').reset_index(drop = True)

    all_prods_control = all_prods_control[all_prods_control['Product'] != 0].reset_index(drop = True)

    #sort our dfs by experiment so they are in the same order now
    all_prods_control = all_prods_control.sort_values('ExperimentId')
    all_prods = all_prods.sort_values('ExperimentId')

    drop_indices = []
    for x, y in enumerate(zip(all_prods_control['ExperimentId'], all_prods_control['Product'])):
        if y not in zip(all_prods['ExperimentId'], all_prods['Product']):
            drop_indices.append(x)
    all_prods_control = all_prods_control.drop(all_prods_control.index[drop_indices]).sort_values('ExperimentId')

    #sort values by BOTH ID and Product
    all_prods = all_prods.sort_values(by = ['ExperimentId', 'Product'])
    all_prods_control = all_prods_control.sort_values(by = ['ExperimentId', 'Product'])
    
    #now we can add in the control group NPS to the correct experiment (there will be no score for exps that didn't have a control group)
    all_prods_control['NPS Control'] = [x for x in all_prods['NPS Control']]

    #calculating MOE for control group
    all_prods_control['MOE Control'] = calculate_MOE(all_prods_control['NPS Control'], all_prods_control['Promoters'], all_prods_control['Passives'], all_prods_control['Detractors'], 
                                                 all_prods_control['Total N'])

    all_prods_control.fillna(0, inplace = True)

    all_prods['MOE Control'] = [x for x in all_prods_control['MOE Control']]

    #creating upper and lower bound calculations for the Control group and adding them as columns to the score df
    all_prods_control['Control Lower Bound'] = all_prods_control['NPS Control'] - all_prods_control['MOE Control']
    all_prods_control['Control Upper Bound'] = all_prods_control['NPS Control'] + all_prods_control['MOE Control']

    all_prods_control = all_prods_control[all_prods_control['Total N'] > 30].reset_index(drop = True)
    
    print('---->---->Scoring Treatment Group---->---->')
    print(' ')

    #filter out for control groups
    all_prods_treatment = df[df['ExperimentGroup'] == 'T'].groupby(['ExperimentId','Product', 'NPS']).agg({'NPS':'size'}).unstack()

    all_prods_treatment.fillna(0, inplace = True)

    #rename columns
    all_prods_treatment.columns = ['Detractors', 'Passives', 'Promoters']

    #create total N size for control groups
    all_prods_treatment['Total Treatment N'] = all_prods_treatment.sum(1)

    all_prods_treatment = all_prods_treatment.fillna(0).reset_index()

    no_tx_groups = []
    for x in list(all_prods.ExperimentId):
        if x not in list(all_prods_treatment.ExperimentId):
            no_tx_groups.append(x)

    new_rows = []
    for x in no_tx_groups:
        row = pd.Series([x,0,0,0,0,0], index=['ExperimentId', 'Product', 'Detractors', 'Passives', 'Promoters', 'Total Treatment N'])
        new_rows.append(row)

    new_rows_df = pd.DataFrame(new_rows)

    #concatenate the new rows (no control group) with the control group df
    all_prods_treatment = pd.concat([all_prods_treatment, new_rows_df], axis = 0).sort_values(by = 'ExperimentId').reset_index(drop = True)

    all_prods_treatment = all_prods_treatment[all_prods_treatment['Product'] != 0].reset_index(drop = True)

    #sort our dfs by experiment so they are in the same order now
    all_prods_treatment = all_prods_treatment.sort_values('ExperimentId')
    

    drop_indices = []
    for x, y in enumerate(zip(all_prods_treatment['ExperimentId'], all_prods_treatment['Product'])):
        if y not in zip(all_prods_control['ExperimentId'], all_prods_control['Product']):
            drop_indices.append(x)
    all_prods_treatment = all_prods_treatment.drop(all_prods_treatment.index[drop_indices]).sort_values('ExperimentId')

    #sort values by BOTH ID and Product
    all_prods = all_prods.sort_values(by = ['ExperimentId', 'Product'])
    all_prods_treatment = all_prods_treatment.sort_values(by = ['ExperimentId', 'Product'])

    #now we can add in the control group NPS to the correct experiment (there will be no score for exps that didn't have a control group)
    all_prods_treatment['NPS Treatment'] = [x for x in all_prods['NPS Treatment']]

    #calculating MOE for control group
    all_prods_treatment['MOE Treatment'] = calculate_MOE(all_prods_treatment['NPS Treatment'], all_prods_treatment['Promoters'], all_prods_treatment['Passives'], all_prods_treatment['Detractors'], 
                                                 all_prods_treatment['Total Treatment N'])

    all_prods_treatment.fillna(0, inplace = True)

    all_prods['MOE Treatment'] = [x for x in all_prods_treatment['MOE Treatment']]

    #creating upper and lower bound calculations for the Control group and adding them as columns to the score df
    all_prods_treatment['Treatment Lower Bound'] = all_prods_treatment['NPS Treatment'] - all_prods_treatment['MOE Treatment']
    all_prods_treatment['Treatment Upper Bound'] = all_prods_treatment['NPS Treatment'] + all_prods_treatment['MOE Treatment']

    all_prods_treatment = all_prods_treatment[all_prods_treatment['Total Treatment N'] > 30].reset_index(drop = True)
    
    all_prods_treatment.sort_values(by = ['ExperimentId', 'Product'], inplace = True)

    all_prods_control.sort_values(by = ['ExperimentId', 'Product'], inplace = True)

    all_prods['Control Lower Bound'] = [x for x in all_prods_control['Control Lower Bound']]
    all_prods['Control Upper Bound'] = [x for x in all_prods_control['Control Upper Bound']]

    all_prods['Treatment Lower Bound'] = [x for x in all_prods_treatment['Treatment Lower Bound']]
    all_prods['Treatment Upper Bound'] = [x for x in all_prods_treatment['Treatment Upper Bound']]
    
    #determine statsig
     #create boolean lists for if the lower and upper bounds of the control were greater/less than those of the treatment group
    a = (all_prods['Control Upper Bound'] > all_prods['Treatment Lower Bound']) & (all_prods['Control Lower Bound'] > all_prods['Treatment Lower Bound'])
    b = (all_prods['Control Upper Bound'] > all_prods['Treatment Upper Bound']) & (all_prods['Control Lower Bound'] > all_prods['Treatment Upper Bound'])
    c = (all_prods['Treatment Upper Bound'] > all_prods['Control Lower Bound']) & (all_prods['Treatment Lower Bound'] > all_prods['Control Lower Bound'])
    d = (all_prods['Treatment Upper Bound'] > all_prods['Control Upper Bound']) & (all_prods['Treatment Lower Bound'] > all_prods['Control Upper Bound'])
    
    #concatenate control and treatment pairs
    control_statsig = pd.concat([a, b], axis = 1)
    treatment_statsig = pd.concat([c, d], axis = 1)
    
    #must have a pair of True values (either two on left, or two on right) to be statsig
    control_statsig = pd.DataFrame(control_statsig.sum(1) == 2).rename(columns = {0:'StatSig?'})
    control_sig_indices = control_statsig[control_statsig['StatSig?'] == True].index
    
    #must have a pair of True values (either two on left, or two on right) to be statsig
    treatment_statsig = pd.DataFrame(treatment_statsig.sum(1) == 2).rename(columns = {0:'StatSig?'})
    treatment_sig_indices = treatment_statsig[treatment_statsig['StatSig?'] == True].index
    
    #create series so that you can concatenate them next into one column then one list
    treatment_sig_indices = pd.Series([x for x in treatment_sig_indices])
    control_sig_indices = pd.Series([x for x in control_sig_indices])
    
    #get statistically significant index numbers to filter out statsig experiments
    sig_indices = [x for x in pd.concat([treatment_sig_indices, control_sig_indices], axis = 0).reset_index(drop = True)]
    
    #run loop to tag StatSig experiments and create a column for their labels
    statsig = []
    for x in all_prods.index:
        if x in sig_indices:
            statsig.append('StatSig')
        else:
            statsig.append('Not')
    
    #create column for statsig in the results
    all_prods['StatSig'] = statsig
    
    #merge in Experiment URL from origininal df
    all_prods = df[['ExperimentId', 'Platform', 'Product', 'ExperimentUrl']].drop_duplicates().merge(all_prods, on = ['ExperimentId', 'Platform', 'Product']) \
        .sort_values('ExperimentId').reset_index(drop = True)
    
    #select only the columns we want to send to a csvand include in report
    all_prods = all_prods[['ExperimentId', 'Platform', 'Product', 'ExperimentUrl', 'TreatmentSize', 'ControlSize', 'NPS Treatment', 'NPS Control', 'Difference (T - C)',
                      'MOE Treatment', 'MOE Control', 'Control Lower Bound', 'Control Upper Bound', 'Treatment Lower Bound', 'Treatment Upper Bound', 'StatSig']]
    
    print(' ')
    print('---------------->Function Executed Successfully<---------------------')
    print('  ')
    print('  ')
    print('*******AB Test Complete**********')
    
    return score_df, all_prods

#call the function to run tests on the suite and app data as well as the entire df overall
suite_platform_df, suite_product_df = run_ab_test(suite_df)
app_platform_df, app_product_df = run_ab_test(app_df)
both_platform_df, both_app_df = run_ab_test(suite_and_app_df)

---->---->Creating and Scoring Data Frame for Experiments Aggregated By Product<----<-----
 
----->----->Scoring Control Group----->----->
 
----->----->Scoring Treatment Group----->----->
 
---->---->Creating and Scoring Each Experiment, Broken Down By Product, Starting With Control Group<----<----
 
---->---->Scoring Control Groups---->---->
 
---->---->Scoring Treatment Group---->---->
 
 
---------------->Function Executed Successfully<---------------------
  
  
*******AB Test Complete**********
---->---->Creating and Scoring Data Frame for Experiments Aggregated By Product<----<-----
 
----->----->Scoring Control Group----->----->
 
----->----->Scoring Treatment Group----->----->
 
---->---->Creating and Scoring Each Experiment, Broken Down By Product, Starting With Control Group<----<----
 
---->---->Scoring Control Groups---->---->
 
---->---->Scoring Treatment Group---->---->
 
 
---------------->Function Executed Successfully<---------------------
  
  
*******AB Test Complet

In [1542]:
#send combined df to csv
both_platform_df.to_csv('aggregated_platform_ab_test.csv', index = False)
both_app_df.to_csv('aggregated_product_ab_test.csv', index = False)

In [1543]:
#send app and suite data to csv files
app_platform_df.to_csv('app_platform_ab_test.csv', index = False)
app_product_df.to_csv('app_product_ab_test.csv', index = False)
suite_platform_df.to_csv('suite_platform_ab_test.csv', index = False)
suite_product_df.to_csv('suite_product_ab_test.csv', index = False)